# Overview
This code analyse data for emotional oddball task in simultaneously collected EEG, fMRI data. Overview of task, and population, and results from analysing a part of data can be found:

* 
* Electrophysiological correlates of fearful and sad distraction on target processing in adolescents with attention deficit-hyperactivity symptoms and affective disorders. 
* Singhal A, Shafer AT, Russell M, Gibson B, Wang L, Vohra S, Dolcos F. 
* Front Integr Neurosci. 2012 Dec 19;6:119. doi: 10.3389/fnint.2012.00119. PMID: 23267319; PMCID: PMC3525949.

## Task 
* Participants performed a modified version of the emotional oddball paradigm (Wang et al.,2005) which consisted of frequent stimuli serving as the baseline[scrambled pictures, 79% (465 trials)],infrequent distracters and odd ball targets, 21% (124 trials).Infrequent distracters consisted of sad and fearful pictures (13 trials each), neutral pictures (26 trials), and positive pictures (4 trials).The oddball targets(circles)were sub-grouped according to their preceding infrequent stimulus type [i.e.,target-after-sad (11 trials), target-after-fear (11 trials), target-after-target (24 trials), andtarget-after-neutral stimuli(22trials)].

* The participants’ task was to indicate whether the stimulus was a target or non-target by pressing a button. Participants were instructed to make a right hand button press any time they saw a target (circle) and a lef thand button press toall other stimuli (i.e., frequent scrambled and infrequent sad, fearful, neutral, and positive distracters). The inital , with targets and distractors. 

## Analysis 
This code extracts peaks and latency of ERP signals in a set of sites, whose electrode number and time window is specified in the code.


Importing packages

In [1]:
%pylab inline
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from collections import namedtuple
from scipy import signal
import numpy as np
import os
from os.path import exists
import sys
from numpy import NaN, Inf, arange, isscalar, asarray, array
import csv



%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
Result = namedtuple('Result', 'subject condition')
#Event = namedtuple('Event', 'timestamp value mean') # Old
Event = namedtuple('Event', 'timestamp value mean status') # New

Defining path, interval, and channels

In [4]:
timeseries_path='C:\\Users\\user\\Documents\\Reyhaneh\\EEG+fMRI\\EO\\EEG_preproc\\ts\\'
# List the folders in the specified directory
ALLERP=['P3','LPP']
CSV_out=timeseries_path+"ERP_measures.csv"


start_time = -300   # ms
sample_interval = 4 # ms

# These are for 256 channel, not 64 as in EEG+fMRI
Cz_channels = (65) #P3
Pz_channels = (34) #LPP

desire_channel_P1 = []
desire_channel_P1name = []

desire_channel_P3 = [Cz_channels] # was added for target in Moore_et al 2019
desire_channel_P3name = ['Cz']#, 'Pz', 'PCz','CL', 'CR', 'PL', 'PR']

desire_channel_LPP = [Pz_channels]#, PL_channels, PR_channels, T5_channels, T6_channels] # was added for distractor in Moore_et al 2019
desire_channel_LPPname = ['Pz']#, 'PL','PR','T5','T6']


P3_start_time, P3_end_time = 250, 550 # Peak 
P1_start_time, P1_end_time = 50, 200 # Peak
LPP_start_time, LPP_end_time = 650, 950 # mean 

In [5]:
def convert_to_channel(x):
    return x + 1

def time_conversion_function(start_time, sample_interval):
    def conversion(x):
        return start_time + x * sample_interval
    return conversion

In [6]:
def average_channel_values(result, channels):
    selected_channels = result.loc[:, channels]
    if isinstance(channels, int):
        return selected_channels
    else:
        return selected_channels.mean(axis=1)

In [7]:
def peakdet(v, delta, x = None):
#peakdetect.py 
#https://gist.github.com/endolith/250860#file-peakdetect-py
    """
    Converted from MATLAB script at http://billauer.co.il/peakdet.html
    Returns two arrays
    function [maxtab, mintab]=peakdet(v, delta, x)
    %PEAKDET Detect peaks in a vector
    %        [MAXTAB, MINTAB] = PEAKDET(V, DELTA) finds the local
    %        maxima and minima ("peaks") in the vector V.
    %        MAXTAB and MINTAB consists of two columns. Column 1
    %        contains indices in V, and column 2 the found values.
    %      
    %        With [MAXTAB, MINTAB] = PEAKDET(V, DELTA, X) the indices
    %        in MAXTAB and MINTAB are replaced with the corresponding
    %        X-values.
    %
    %        A point is considered a maximum peak if it has the maximal
    %        value, and was preceded (to the left) by a value lower by
    %        DELTA.
    
    % Eli Billauer, 3.4.05 (Explicitly not copyrighted).
    % This function is released to the public domain; Any use is allowed.
    """

    maxtab = []
    mintab = []
    if x is None:
        x = arange(len(v))
    v = asarray(v)
    if len(v) != len(x):
        sys.exit('Input vectors v and x must have same length')
    if not isscalar(delta):
        sys.exit('Input argument delta must be a scalar')
    if delta <= 0:
        sys.exit('Input argument delta must be positive')
    mn, mx = Inf, -Inf
    mnpos, mxpos = NaN, NaN
    lookformax = True
    for i in arange(len(v)):
        this = v[i]
        if this > mx:
            mx = this
            mxpos = x[i]
        if this < mn:
            mn = this
            mnpos = x[i]
        if lookformax:
            if this < mx-delta:
                maxtab.append((mxpos, mx))
                mn = this
                mnpos = x[i]
                lookformax = False
        else:
            if this > mn+delta:
                mintab.append((mnpos, mn))
                mx = this
                mxpos = x[i]
                lookformax = True
    return array(maxtab), array(mintab)


In [8]:
def find_maximum_and_mean_value_in_time_window(time_series, start_time_In, end_time_In):
#     from matplotlib.pyplot import plot, scatter, show
    
    ExtensionTimeLeft = 5 #ms
    ExtensionTimeRight = 5 #ms
    peak_res = .01 
    
    # Baseline noise levvel
    time_slice = time_series.loc[-300-1:0]
    baseline_std = time_slice.std()
    baseline_max = abs(time_slice).max()
    # print('Baseline std:',baseline_std,'& Baseilne Max:',baseline_max)
    # Ideas: check baseline_std, if it is too noisy: discard this case
    # check max_basesline, Peak should be above that
    # check the slop of the waveform, it should be more than 25, 
    # check peak amplitude, it should be more than thr

    
    time_slice = time_series.loc[start_time_In-1:end_time_In]
    start_time = time_slice.index[0]
    end_time = time_slice.index[-1]
#     print(start_time,end_time)
    maxtab, mintab = peakdet(time_slice,peak_res)
    if len(array(maxtab))>0:
        myindexmax = [int(x) for x in floor(array(maxtab)[:,0])]
#         scatter(time_slice.index[myindexmax], array(maxtab)[:,1], color='blue',marker='x')
        LeftMax=array([time_slice.index[myindexmax], array(maxtab)[:,1]])
    else: 
        LeftMax = np.array([(),()]) 
        
    time_slice_rev = time_series.loc[end_time:start_time:-1]
    maxtab_rev, mintab_rev = peakdet(time_slice_rev,peak_res)
    if len(array(maxtab_rev))>0:
        myindexmax_rev = [int(x) for x in floor(array(maxtab_rev)[:,0])]
#         scatter(time_slice_rev.index[myindexmax_rev], array(maxtab_rev)[:,1], color='cyan',marker='x')
        RightMax=array([time_slice_rev.index[myindexmax_rev], maxtab_rev[:,1]])
    else: 
        RightMax = np.array([(),()])  
        
#     print('888888888888888888888888888888888888888888888888888888888888')
#     print(time_slice)
#     print(time_slice_rev)
#     print('maxtab =',maxtab)
#     print('maxtab_rev =',maxtab_rev)

#     plot(time_slice)
    maxtabConc=np.concatenate((LeftMax,RightMax),axis=1)
#     print('LeftMax=',LeftMax)
#     print('RightMax=',RightMax)
#     print('maxtabConc=',maxtabConc)
    
    maxLocalInd = maxtabConc[1,:].argmax()
    maxLocalMax = maxtabConc[1,maxLocalInd];
    maxLocalIndOut = maxtabConc[0,maxLocalInd] 
#     print('maxLocalMax = ',maxLocalMax)
#     print('maxLocalInd = ',maxtabConc[0,maxLocalInd])

#     print('tmp:',maxtabConc[0,maxLocalInd])
#     print('tt:',time_slice.index[0])
    if maxLocalIndOut == start_time: # Peak at start point
        # Extend to the left and look for the first peak
        time_sliceExt_rev = time_series.loc[start_time:start_time-ExtensionTimeLeft:-1]  
#         plot(time_sliceExt_rev)
        maxtabExt_rev, mintabExt_rev = peakdet(time_sliceExt_rev,peak_res)
        if len(array(maxtabExt_rev))>0: # peak was found by extending to the left
            maxLocalMax = maxtabExt_rev[0,1]
            myindexmaxExt_rev = [int(x) for x in floor(array(maxtabExt_rev)[:,0])]
            maxLocalIndOut = time_sliceExt_rev.index[myindexmaxExt_rev[0]]
#             print('haha!! myindexmaxExt_rev=',myindexmaxExt_rev)
#             scatter(time_sliceExt_rev.index[myindexmaxExt_rev[0]], array(maxtabExt_rev)[0,1], color='purple',marker='o')
            Status = 'L'
        else: # peak was NOT found by extending to the left 
            # check again for the local max in the main area
            LeftMax = np.delete(LeftMax, 0, axis=1) # remove the first column which was left boundry max
            maxtabConc = np.concatenate((LeftMax,RightMax),axis=1)
            if len(array(maxtabConc[1,:]))>0:
                maxLocalInd = maxtabConc[1,:].argmax()
                maxLocalMax = maxtabConc[1,maxLocalInd];
                maxLocalIndOut = maxtabConc[0,maxLocalInd] 
    #             print('tmp,maxlocalMax=',maxLocalMax)
    #             print('boundry=',time_series.loc[end_time:end_time+sample_interval-1].mean())
                if  maxLocalIndOut == end_time:# Peak at end point:
                    # Extend to the right and look for peak
                    time_sliceExt = time_series.loc[end_time:end_time+ExtensionTimeRight]
    #                 plot(time_sliceExt)
                    maxtabExt, mintabExt = peakdet(time_sliceExt,peak_res)
                    if len(array(maxtabExt))>0:
                        maxLocalMax = maxtabExt[0,1]            
                        myindexmaxExt = [int(x) for x in floor(array(maxtabExt)[:,0])]
                        maxLocalIndOut = time_sliceExt.index[myindexmaxExt[0]]
    #                     scatter(time_sliceExt.index[myindexmaxExt], array(maxtabExt)[:,1], color='purple',marker='o')
                        Status = 'R'
                    else: 
                        # check again for the local max in the boundry
                        RightMax = np.delete(RightMax, 0, axis=1) # remove the first column which was right boundry max
    #                     print(RightMax)
                        maxtabConc=np.concatenate((LeftMax,RightMax),axis=1)
    #                     print(maxtabConc)
                        if len(array(maxtabConc[1,:]))>0:
                            maxLocalInd = maxtabConc[1,:].argmax()
                            maxLocalMax = maxtabConc[1,maxLocalInd];
                            maxLocalIndOut = maxtabConc[0,maxLocalInd] 
    #                         scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
                            Status = 'G'
                        else:
                            #Nothing is found!
                            maxLocalMax = 999
                            maxLocalIndOut = 999
                            Status = 'NaN'            
                else: 
    #                 scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
                    Status = 'G'
            else:
                #Nothing is found!
                maxLocalMax = 999
                maxLocalIndOut = 999
                Status = 'NaN'                                 
            
    elif maxLocalIndOut == end_time:# Peak at end point
        # Extend to the right and look for the first peak
        
        time_sliceExt = time_series.loc[end_time:end_time+ExtensionTimeRight]
#         plot(time_sliceExt)
        maxtabExt, mintabExt = peakdet(time_sliceExt,peak_res)
        if len(array(maxtabExt))>0: # peak was found by extending to the right
            maxLocalMax = maxtabExt[0,1]            
            myindexmaxExt = [int(x) for x in floor(array(maxtabExt)[:,0])]
            maxLocalIndOut = time_sliceExt.index[myindexmaxExt[0]]
#             print('haha!! myindexmaxExt=',myindexmaxExt)
#             scatter(time_sliceExt.index[myindexmaxExt[0]], array(maxtabExt)[0,1], color='purple',marker='o')
            Status = 'R'
        else: # peak was NOT found by extending to the right 
            # check again for the local max in the main area
            RightMax = np.delete(RightMax, 0, axis=1) # remove the first column which was right boundry max
            # check again for the local max in the boundry
            maxtabConc = np.concatenate((LeftMax,RightMax),axis=1)
#             print('RightMaxNew=',RightMax)

            if len(array(maxtabConc[1,:]))>0:
                maxLocalInd = maxtabConc[1,:].argmax()
                maxLocalMax = maxtabConc[1,maxLocalInd];
                maxLocalIndOut = maxtabConc[0,maxLocalInd] 
    #             print('tmp,maxlocalMax=',maxLocalMax)
    #             print('boundry=',time_series.loc[start_time:start_time+sample_interval-1].mean())
                if maxLocalIndOut == time_slice.index[0]:# Peak at start point:
                    # Extend to the left and look for the first peak
                    time_sliceExt_rev = time_series.loc[start_time:start_time-ExtensionTimeLeft:-1]   
    #                 plot(time_sliceExt_rev)
                    maxtabExt_rev, mintabExt_rev = peakdet(time_sliceExt_rev,peak_res)                
                    if len(array(maxtabExt_rev))>0:
                        maxLocalMax = maxtabExt_rev[0,1]            
                        myindexmaxExt = [int(x) for x in floor(array(maxtabExt_rev)[:,0])]
                        maxLocalIndOut = time_sliceExt_rev.index[myindexmaxExt[0]]
    #                     scatter(time_sliceExt_rev.index[myindexmaxExt], array(maxtabExt_rev)[:,1], color='purple',marker='o')
                        Status = 'L'
                    else:
                        # check again for the local max in the boundry
                        LeftMax = np.delete(LeftMax, 0, axis=1) # remove the first column which was left boundry max
    #                     print(LeftMax)
                        maxtabConc=np.concatenate((LeftMax,RightMax),axis=1)
    #                     print(maxtabConc)
                        if len(array(maxtabConc[1,:]))>0:
                            maxLocalInd = maxtabConc[1,:].argmax()
                            maxLocalMax = maxtabConc[1,maxLocalInd];
                            maxLocalIndOut = maxtabConc[0,maxLocalInd] 
    #                         scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
                            Status = 'G'
                        else:
                            #Nothing is found!
                            maxLocalMax = 999
                            maxLocalIndOut = 999
                            Status = 'NaN'            
                else: 
    #                 scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
                    Status = 'G'        
            else:
                #Nothing is found!
                maxLocalMax = 999
                maxLocalIndOut = 999
                Status = 'NaN'                
    else:
#         scatter(maxtabConc[0,maxLocalInd], maxLocalMax, color='purple',marker='o')
        Status = 'G'
#     show()
#     print(time_series.loc[start_time:start_time+sample_interval-1])
#     print('1st sample =',time_series.loc[start_time:start_time+sample_interval-1].mean())
#     print('Overal Max =',maxLocalMax)
#     print(Status)
    
       
    #return Event(timestamp=time_slice.idxmax(), value=time_slice.max(), mean=time_slice.mean()) # old
    return Event(timestamp=int(maxLocalIndOut), value=maxLocalMax, mean=time_slice.mean(), status=Status) # new

In [9]:
def save_figure(waveform, experiment, channel_name, ERP, event, start_time, end_time):
    title_template = '{subject}|{condition}|{channel_name}|{ERP_peak}|{mean_amp}|{event_time}|{event_value}|{Status}'
    title = title_template.format(
        subject=experiment.subject,
        condition=experiment.condition,
        channel_name=channel_name,
        ERP_peak=ERP,
        mean_amp="{:.4f}".format(event.mean),
        event_time=event.timestamp,
        event_value="{:.4f}".format(event.value),
        Status=event.status,
    )
    filename_template = '{subject}_{condition}_{channel_name}_{ERP_peak}_{Status}.png'
    filename = filename_template.format(
        subject=experiment.subject,
        condition=experiment.condition,
        channel_name=channel_name,
        ERP_peak=ERP,
        Status=event.status,
    )
    print(title)
#    print (filename)
    plot_axes = waveform.plot(title=title)

#     plot_axes.axvline(event.timestamp)
#     plot_axes.axhline(event.value)
    plot_axes.scatter(event.timestamp, event.value, color='red',marker='o')
    plot_axes.axvline(start_time, color='k',ls='dashed')
    plot_axes.axvline(end_time, color='k',ls='dashed')

    fig = plot_axes.get_figure()
    fig.savefig(filename)
    fig.clear()


In [10]:
def save_figure_Both(waveform_All, result_map_keys, channel_name, ERP, start_time, start_time_2, end_time, end_time_2):
    title_template = '{subject}|{condition}|{channel_name}|{ERP_peak}'
    title = title_template.format(
        subject=list(result_map_keys)[0].subject,
        condition=list(result_map_keys)[0].condition,
        channel_name=channel_name,
        ERP_peak=ERP,
        
    )
    filename_template = '{subject}_{condition}_{channel_name}_{ERP_peak}.png'
    filename = filename_template.format(
        subject=list(result_map_keys)[0].subject,
        condition=list(result_map_keys)[0].condition,
        channel_name=channel_name,
        ERP_peak=ERP,
    )
    print(title)

    for result in sorted(result_map_keys):
        plot_axes = Waveform_All[result].plot(label=result.condition,title=title)
        
    plot_axes.legend(loc="lower left")
    plot_axes.axvline(start_time, color='k',ls='dashed')
    plot_axes.axvline(end_time, color='k',ls='dashed')
    plot_axes.axvline(start_time_2, color='k',ls='dashed')
    plot_axes.axvline(end_time_2, color='k',ls='dashed')
    
    #make area grey

    fig = plot_axes.get_figure()
    print(filename)
    fig.savefig(filename)
    fig.clear()


In [11]:
def list_file_folders(directory,file_folder):
    folders = []
    for item in os.listdir(directory):
        if file_folder=='file':
            if os.path.isfile(os.path.join(directory, item)):
                folders.append(item)
        elif file_folder=='folder':
            if os.path.isdir(os.path.join(directory, item)):
                folders.append(item)
    return folders

In [12]:
# Extract ERP measure for each subject & condition
with open(CSV_out, mode='w', newline='') as file:
    writer = csv.writer(file)
    Header_row=['Subject_ID','Run','Condition','Site','ERP','Mean','Peak','Latency']
    writer.writerow(Header_row)
        
    Subj_list = list_file_folders(timeseries_path,'folder')
    #Subj_list = ['AB_04152014']
    for Subj in Subj_list:
        #print(Subj)
        Run_list=list_file_folders(timeseries_path+Subj+'\\','folder')
        for Run in Run_list:
        #    print('   '+Run)
            file_list=list_file_folders(timeseries_path+Subj+'\\'+Run+'\\','file')
            for ts_file in file_list:
                if exists(timeseries_path+Subj+'\\'+Run+'\\'+ts_file):
                    #print(ts_file)
                    
                    result_map = {}
                    filename=timeseries_path+Subj+'\\'+Run+'\\'+ts_file
                    raw_data = pd.read_csv(filename, header=None)
                    res = Result(Subj,ts_file[:-4]) 
                    #   print(res)
                    raw_data = pd.read_csv(filename, header=None)
                    cleaned_data = raw_data.rename(index=time_conversion_function(start_time, sample_interval), columns=convert_to_channel)
                    result_map[res] = cleaned_data
                    for MyERP in ALLERP:
                        desire_channel_ERP_name=getattr(sys.modules[__name__], f"desire_channel_{MyERP}name")
                        desire_channel_ERP=getattr(sys.modules[__name__], f"desire_channel_{MyERP}")
                        ERP_start_time=ERP=getattr(sys.modules[__name__], f"{MyERP}_start_time")
                        ERP_end_time=ERP=getattr(sys.modules[__name__], f"{MyERP}_end_time")

                        for i_ch,mychannelname in enumerate(desire_channel_ERP_name):
                            mychannel=desire_channel_ERP[i_ch]
                            #print(i_ch)
                            #print(ch_value)
                            #print(mychannel)

                            waveform = average_channel_values(result_map[res], mychannel) #open waveforms for all channels for one participant, averaged

                            maximal_event = find_maximum_and_mean_value_in_time_window(waveform, ERP_start_time, ERP_end_time)
                            print(res.subject+'|'+Run+'|'+ts_file[:-4]+'|'+mychannelname+'|'+MyERP+'|'+"{:.4f}".format(maximal_event.mean)+'|'+"{:.4f}".format(maximal_event.value)+'|'+"{:.0f}".format(maximal_event.timestamp))
                            row_val=[res.subject,Run,ts_file[:-4],mychannelname,MyERP,maximal_event.mean,maximal_event.value,maximal_event.timestamp]
                            writer.writerow(row_val)

AB_04152014|run01|Neg_1|Cz|P3|-0.9677|12.8120|432
AB_04152014|run01|Neg_1|Pz|LPP|1.5843|13.3260|796
AB_04152014|run01|Neg_2|Cz|P3|6.6058|25.9720|424
AB_04152014|run01|Neg_2|Pz|LPP|-3.6351|18.6480|652
AB_04152014|run01|Neg_3|Cz|P3|10.1255|26.3840|440
AB_04152014|run01|Neg_3|Pz|LPP|0.6735|18.2410|696
AB_04152014|run01|Neg_4|Cz|P3|8.1579|26.5940|300
AB_04152014|run01|Neg_4|Pz|LPP|-2.0497|13.3620|652
AB_04152014|run01|Neg_5|Cz|P3|9.6300|21.6040|408
AB_04152014|run01|Neg_5|Pz|LPP|2.2951|11.2710|704
AB_04152014|run01|Neg_6|Cz|P3|5.6577|15.9310|424
AB_04152014|run01|Neg_6|Pz|LPP|4.5941|16.2210|936
AB_04152014|run01|Neg_7|Cz|P3|10.5451|22.2790|440
AB_04152014|run01|Neg_7|Pz|LPP|1.8661|17.1050|652
AB_04152014|run01|Neu_1|Cz|P3|-4.1598|9.8755|500
AB_04152014|run01|Neu_1|Pz|LPP|0.6697|14.5500|696
AB_04152014|run01|Neu_2|Cz|P3|10.1658|24.0950|500
AB_04152014|run01|Neu_2|Pz|LPP|-9.4867|-1.3397|884
AB_04152014|run01|Neu_3|Cz|P3|22.0934|29.3510|268
AB_04152014|run01|Neu_3|Pz|LPP|-8.3858|1.2408|768
AB

AB_04152014|run04|Neu_1|Pz|LPP|1.9702|17.5480|672
AB_04152014|run04|Neu_2|Cz|P3|-3.9486|8.2032|288
AB_04152014|run04|Neu_2|Pz|LPP|1.8639|21.8240|784
AB_04152014|run04|Neu_3|Cz|P3|12.0538|26.3050|444
AB_04152014|run04|Neu_3|Pz|LPP|6.4911|19.4640|868
AB_04152014|run04|Neu_4|Cz|P3|-1.3300|7.5514|276
AB_04152014|run04|Neu_4|Pz|LPP|3.0323|12.8260|720
AB_04152014|run04|Neu_5|Cz|P3|13.7227|31.4900|436
AB_04152014|run04|Neu_5|Pz|LPP|3.8621|23.7620|720
AB_04152014|run04|Neu_6|Cz|P3|2.9468|18.6950|440
AB_04152014|run04|Neu_6|Pz|LPP|-8.4616|-0.4956|796
AB_04152014|run04|Neu_7|Cz|P3|-3.8379|8.4275|304
AB_04152014|run04|Neu_7|Pz|LPP|10.4907|17.2290|680
AB_04152014|run04|Neu_8|Cz|P3|5.4712|21.6970|484
AB_04152014|run04|Neu_8|Pz|LPP|-1.8386|5.3500|720
AB_04152014|run04|Targ_1|Cz|P3|-3.6026|24.1290|348
AB_04152014|run04|Targ_1|Pz|LPP|-8.8300|11.9460|748
AB_04152014|run04|Targ_10|Cz|P3|0.1568|20.3790|392
AB_04152014|run04|Targ_10|Pz|LPP|-10.6399|21.5600|900
AB_04152014|run04|Targ_11|Cz|P3|12.3526|27.79

AM_05082014|run02|Neg_8|Cz|P3|-4.5812|43.2630|512
AM_05082014|run02|Neg_8|Pz|LPP|-3.8074|22.0170|732
AM_05082014|run02|Neu_1|Cz|P3|-18.4274|10.9740|492
AM_05082014|run02|Neu_1|Pz|LPP|-6.4252|8.3525|820
AM_05082014|run02|Neu_2|Cz|P3|-1.2402|54.3630|456
AM_05082014|run02|Neu_2|Pz|LPP|1.3068|15.9900|876
AM_05082014|run02|Neu_3|Cz|P3|-49.0040|65.2520|504
AM_05082014|run02|Neu_3|Pz|LPP|10.5836|33.0640|896
AM_05082014|run02|Neu_4|Cz|P3|-18.1801|16.8120|452
AM_05082014|run02|Neu_4|Pz|LPP|1.5088|20.1140|676
AM_05082014|run02|Neu_5|Cz|P3|-8.1722|38.4440|404
AM_05082014|run02|Neu_5|Pz|LPP|-1.3895|10.0520|808
AM_05082014|run02|Neu_6|Cz|P3|4.9292|28.3820|268
AM_05082014|run02|Neu_6|Pz|LPP|-11.9881|11.5500|920
AM_05082014|run02|Neu_7|Cz|P3|-11.4901|14.6520|516
AM_05082014|run02|Neu_7|Pz|LPP|-6.3380|8.3001|664
AM_05082014|run02|Neu_8|Cz|P3|10.5834|52.3610|484
AM_05082014|run02|Neu_8|Pz|LPP|-4.3084|15.5650|668
AM_05082014|run02|Targ_1|Cz|P3|-12.5983|19.2090|392
AM_05082014|run02|Targ_1|Pz|LPP|16.8936

AM_05082014|run05|Neu_2|Cz|P3|-10.2729|16.1280|300
AM_05082014|run05|Neu_2|Pz|LPP|-1.3082|11.1330|736
AM_05082014|run05|Neu_3|Cz|P3|-6.7444|24.9820|400
AM_05082014|run05|Neu_3|Pz|LPP|-5.2382|16.7110|796
AM_05082014|run05|Neu_4|Cz|P3|4.6445|77.2960|496
AM_05082014|run05|Neu_4|Pz|LPP|-1.7212|12.3650|676
AM_05082014|run05|Neu_5|Cz|P3|2.4125|77.3380|460
AM_05082014|run05|Neu_5|Pz|LPP|-4.1863|15.3380|680
AM_05082014|run05|Neu_6|Cz|P3|-1.9727|44.7830|308
AM_05082014|run05|Neu_6|Pz|LPP|-8.6464|4.1161|924
AM_05082014|run05|Neu_7|Cz|P3|-14.5569|68.1620|424
AM_05082014|run05|Neu_7|Pz|LPP|-4.5915|7.0879|712
AM_05082014|run05|Neu_8|Cz|P3|-2.3365|34.4390|468
AM_05082014|run05|Neu_8|Pz|LPP|-1.4881|14.8850|900
AM_05082014|run05|Targ_1|Cz|P3|0.0621|39.3310|436
AM_05082014|run05|Targ_1|Pz|LPP|6.2965|17.7620|656
AM_05082014|run05|Targ_10|Cz|P3|2.0433|29.7280|408
AM_05082014|run05|Targ_10|Pz|LPP|0.5010|10.7970|700
AM_05082014|run05|Targ_11|Cz|P3|-12.2431|38.6600|380
AM_05082014|run05|Targ_11|Pz|LPP|11.59

CJ_06052014|run03|Neu_6|Cz|P3|1.9657|12.7450|424
CJ_06052014|run03|Neu_6|Pz|LPP|-7.1665|4.3165|708
CJ_06052014|run03|Neu_7|Cz|P3|4.7884|15.4150|544
CJ_06052014|run03|Neu_7|Pz|LPP|-0.8119|11.1120|668
CJ_06052014|run03|Neu_8|Cz|P3|-1.9092|11.0740|420
CJ_06052014|run03|Neu_8|Pz|LPP|-0.2809|7.1448|672
CJ_06052014|run03|Targ_1|Cz|P3|-4.1905|9.2530|296
CJ_06052014|run03|Targ_1|Pz|LPP|-11.1708|-0.6487|756
CJ_06052014|run03|Targ_10|Cz|P3|-0.0192|13.4890|380
CJ_06052014|run03|Targ_10|Pz|LPP|-7.0228|1.8829|908
CJ_06052014|run03|Targ_11|Cz|P3|4.1308|15.3010|488
CJ_06052014|run03|Targ_11|Pz|LPP|-2.9322|8.6805|776
CJ_06052014|run03|Targ_12|Cz|P3|1.7016|11.0810|364
CJ_06052014|run03|Targ_12|Pz|LPP|-7.6882|5.4209|908
CJ_06052014|run03|Targ_2|Cz|P3|0.2767|11.6970|316
CJ_06052014|run03|Targ_2|Pz|LPP|-6.3226|7.5099|860
CJ_06052014|run03|Targ_3|Cz|P3|5.5119|18.1830|388
CJ_06052014|run03|Targ_3|Pz|LPP|-2.9575|5.1563|660
CJ_06052014|run03|Targ_4|Cz|P3|5.4205|15.4660|312
CJ_06052014|run03|Targ_4|Pz|LPP|-0.3

...

TD_05012014|run02|Neu_4|Cz|P3|7.3782|19.0280|452
TD_05012014|run02|Neu_4|Pz|LPP|-0.5643|10.5450|848
TD_05012014|run02|Neu_5|Cz|P3|-2.3972|10.5240|480
TD_05012014|run02|Neu_5|Pz|LPP|-1.1553|5.2860|688
TD_05012014|run02|Neu_6|Cz|P3|-7.4701|4.3434|276
TD_05012014|run02|Neu_6|Pz|LPP|-2.0951|4.3376|812
TD_05012014|run02|Neu_7|Cz|P3|3.2525|13.7370|348
TD_05012014|run02|Neu_7|Pz|LPP|-9.0040|0.4701|788
TD_05012014|run02|Neu_8|Cz|P3|-0.4097|12.3420|432
TD_05012014|run02|Neu_8|Pz|LPP|-1.3992|10.1150|712
TD_05012014|run02|Targ_1|Cz|P3|9.5313|25.8600|356
TD_05012014|run02|Targ_1|Pz|LPP|1.6319|14.1340|916
TD_05012014|run02|Targ_10|Cz|P3|9.9854|26.7920|436
TD_05012014|run02|Targ_10|Pz|LPP|0.0818|8.3612|828
TD_05012014|run02|Targ_11|Cz|P3|-3.4404|7.9590|488
TD_05012014|run02|Targ_11|Pz|LPP|0.4470|17.4100|708
TD_05012014|run02|Targ_12|Cz|P3|4.5343|26.9340|388
TD_05012014|run02|Targ_12|Pz|LPP|-6.0592|6.8005|756
TD_05012014|run02|Targ_2|Cz|P3|9.3620|23.6840|320
TD_05012014|run02|Targ_2|Pz|LPP|2.4654|11.

TD_05012014|run05|Neu_4|Cz|P3|-7.7137|3.5852|524
TD_05012014|run05|Neu_4|Pz|LPP|3.6464|13.3850|804
TD_05012014|run05|Neu_5|Cz|P3|1.1337|13.2050|440
TD_05012014|run05|Neu_5|Pz|LPP|9.0844|17.5550|904
TD_05012014|run05|Neu_6|Cz|P3|-4.9843|8.4939|496
TD_05012014|run05|Neu_6|Pz|LPP|-0.7005|7.6613|732
TD_05012014|run05|Neu_7|Cz|P3|-2.3044|5.8978|392
TD_05012014|run05|Neu_7|Pz|LPP|1.7536|8.3669|672
TD_05012014|run05|Neu_8|Cz|P3|3.3799|19.5800|508
TD_05012014|run05|Neu_8|Pz|LPP|1.5521|7.7588|712
TD_05012014|run05|Targ_1|Cz|P3|1.3991|9.9586|308
TD_05012014|run05|Targ_1|Pz|LPP|0.0825|7.1344|724
TD_05012014|run05|Targ_10|Cz|P3|-2.0886|17.9290|368
TD_05012014|run05|Targ_10|Pz|LPP|-5.0616|11.2650|760
TD_05012014|run05|Targ_11|Cz|P3|0.6561|8.9954|472
TD_05012014|run05|Targ_11|Pz|LPP|-0.3428|9.6454|696
TD_05012014|run05|Targ_12|Cz|P3|3.7651|18.4840|404
TD_05012014|run05|Targ_12|Pz|LPP|-0.9517|7.2463|776
TD_05012014|run05|Targ_2|Cz|P3|25.7236|37.4050|332
TD_05012014|run05|Targ_2|Pz|LPP|11.6801|23.1290

TH_12172013|run04|Neu_3|Cz|P3|6.7333|43.6160|296
TH_12172013|run04|Neu_3|Pz|LPP|-1.9402|13.9030|912
TH_12172013|run04|Neu_4|Cz|P3|5.9642|48.0790|444
TH_12172013|run04|Neu_4|Pz|LPP|27.9961|43.8580|940
TH_12172013|run04|Neu_5|Cz|P3|6.4422|33.0010|392
TH_12172013|run04|Neu_5|Pz|LPP|-4.3115|10.9380|888
TH_12172013|run04|Targ_1|Cz|P3|-23.2834|13.7530|256
TH_12172013|run04|Targ_1|Pz|LPP|-5.7334|22.7210|940
TH_12172013|run04|Targ_2|Cz|P3|5.4316|27.0850|344
TH_12172013|run04|Targ_2|Pz|LPP|3.2357|16.1260|736
TH_12172013|run04|Targ_3|Cz|P3|-4.3188|19.0550|384
TH_12172013|run04|Targ_3|Pz|LPP|-10.1559|-0.7581|916
TH_12172013|run04|Targ_4|Cz|P3|16.3780|42.1830|384
TH_12172013|run04|Targ_4|Pz|LPP|13.2986|28.7640|728
TH_12172013|run04|Targ_5|Cz|P3|10.2820|43.6840|300
TH_12172013|run04|Targ_5|Pz|LPP|0.1200|7.8994|732
TH_12172013|run04|Targ_6|Cz|P3|5.4185|32.9720|420
TH_12172013|run04|Targ_6|Pz|LPP|-4.3955|3.6517|820
TH_12172013|run04|Targ_7|Cz|P3|5.8602|31.7180|288
TH_12172013|run04|Targ_7|Pz|LPP|6.29

TH_12172013|run08|Targ_7|Cz|P3|14.0420|30.1590|348
TH_12172013|run08|Targ_7|Pz|LPP|2.2163|23.4190|792
TH_12172013|run08|Targ_8|Cz|P3|-1.6079|9.2029|316
TH_12172013|run08|Targ_8|Pz|LPP|-2.7583|8.0052|860
TH_12172013|run08|Targ_9|Cz|P3|0.0152|39.6760|544
TH_12172013|run08|Targ_9|Pz|LPP|-7.9586|10.3950|764
TP_05162014|run01|Neg_1|Cz|P3|-7.4301|4.6393|320
TP_05162014|run01|Neg_1|Pz|LPP|-8.5463|7.7812|860
TP_05162014|run01|Neg_2|Cz|P3|1.4484|12.7740|452
TP_05162014|run01|Neg_2|Pz|LPP|7.1016|17.7270|888
TP_05162014|run01|Neg_3|Cz|P3|6.2695|14.2200|396
TP_05162014|run01|Neg_3|Pz|LPP|-1.1444|8.3881|864
TP_05162014|run01|Neg_4|Cz|P3|-8.3626|-2.2722|404
TP_05162014|run01|Neg_4|Pz|LPP|-13.5754|-4.5011|844
TP_05162014|run01|Neg_5|Cz|P3|-2.4430|9.2483|508
TP_05162014|run01|Neg_5|Pz|LPP|-8.7917|4.0830|656
TP_05162014|run01|Neg_6|Cz|P3|4.3898|17.7010|404
TP_05162014|run01|Neg_6|Pz|LPP|41.4440|56.7900|688
TP_05162014|run01|Neg_7|Cz|P3|7.5098|24.2350|508
TP_05162014|run01|Neg_7|Pz|LPP|-9.2810|-2.7573|7

TP_05162014|run03|Targ_6|Pz|LPP|4.9317|13.5980|940
TP_05162014|run03|Targ_7|Cz|P3|1.6231|12.8900|256
TP_05162014|run03|Targ_7|Pz|LPP|1.1528|11.5410|752
TP_05162014|run03|Targ_8|Cz|P3|3.8271|10.0870|316
TP_05162014|run03|Targ_8|Pz|LPP|0.8261|10.3030|792
TP_05162014|run03|Targ_9|Cz|P3|-1.0437|9.2771|344
TP_05162014|run03|Targ_9|Pz|LPP|-6.7750|1.2523|764
TP_05162014|run04|Neg_1|Cz|P3|2.4201|11.2850|304
TP_05162014|run04|Neg_1|Pz|LPP|-13.3101|-0.1657|704
TP_05162014|run04|Neg_2|Cz|P3|0.1352|11.4080|508
TP_05162014|run04|Neg_2|Pz|LPP|3.6179|14.5890|692
TP_05162014|run04|Neg_3|Cz|P3|-3.7366|2.9506|396
TP_05162014|run04|Neg_3|Pz|LPP|6.1787|22.0650|800
TP_05162014|run04|Neg_4|Cz|P3|-1.6730|5.1932|544
TP_05162014|run04|Neg_4|Pz|LPP|0.3555|20.5790|672
TP_05162014|run04|Neg_5|Cz|P3|1.9138|10.6440|476
TP_05162014|run04|Neg_5|Pz|LPP|-20.5969|-5.0502|676
TP_05162014|run04|Neg_6|Cz|P3|-2.5821|5.3190|256
TP_05162014|run04|Neg_6|Pz|LPP|-8.5876|2.8067|660
TP_05162014|run04|Neg_7|Cz|P3|4.4531|13.4960|344